In [1]:
from bs4 import BeautifulSoup
import re
import requests

import pandas as pd 

import spacy
import spacy.cli

In [2]:
from time import sleep
from random import randint

In [3]:
dic_sentimentos = {}
lista_sent = [('alegria','alegria', 0), ('tristeza','tristeza', 0), ('raiva','raiva', 0), ('nojo','nojo', 0), ('medo','medo', 0)]
#lista_sent = [('raiva','raiva', 0), ('nojo','nojo', 0), ('medo','medo', 0)]

In [4]:
#aux = 'maca'
#while (lista_sent != [] and aux != 'medo'):
#    print('lala')

In [5]:
def balancear_sentimentos():
    #print('balancear chegou')
    sent_desbalanceado = 'alegria'
    m_desbalanceado = pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()[0]

    for i in range(1,5):
        v = pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()[i]
        
        if v < m_desbalanceado:
            m_desbalanceado = v
            if i == 1:
                sent_desbalanceado = 'tristeza'
            elif i == 2:
                sent_desbalanceado = 'raiva'
            elif i == 3:
                sent_desbalanceado = 'nojo'
            elif i == 4:
                sent_desbalanceado = 'medo'
                
    #print('balancear saida')
    return sent_desbalanceado

In [6]:
def busca_elemento_lista(sentimento):
    #print('busca chegou')
    global lista_sent
    
    sent_origem = 'nulo'
    i = 0
    
    while (i > len(lista_sent) and sent_origem != sentimento):
        s, sent_origem, h = lista_sent[i]
        i += 1
    #print('busca saida')
    return i-1    

In [7]:
def cria_dicionario():
    global lista_sent
    
    if len(dic_sentimentos) < 10:
        s, o, h = lista_sent.pop(0)
    else:        
        sd = balancear_sentimentos()
        indice = busca_elemento_lista(sd)
        s, o, h = lista_sent.pop(indice)

    #verifica se a palavra está no dicionario
    if s in dic_sentimentos:
        return
    
    if h > d:
        dic_sentimentos[s] =  o
        return
    
    #caso as verificações estejam ok, adiciona no dicionário e faz a busca 
    
    dic_sentimentos[s] =  o
    
    url = "https://www.sinonimos.com.br/" + s
    soup = BeautifulSoup(requests.get(url).text, "html5lib")
    retorno = soup.find_all('a', class_ ='sinonimo')
    lista_sent = lista_sent + list(pd.Series(retorno).apply(lambda x: (str(x).split('>')[1][:-3], o, h+1)))
    
    #sleep(randint(1,3))
    
    cria_dicionario() 

In [8]:
d = 3

while lista_sent != []:
    cria_dicionario()

/home/karine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [9]:
(pd.DataFrame.from_dict(dic_sentimentos, orient='index')).to_csv('dic_sentimento_3_niveis_atualizado.txt', sep='\t' )

In [10]:
pd.DataFrame.from_dict(dic_sentimentos, orient='index', columns = ['sent'])['sent'].value_counts()

nojo        1248
alegria     1061
medo         438
raiva        334
tristeza     286
Name: sent, dtype: int64